In [4]:
import pandas as pd

from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options

import concurrent.futures 

from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup as bs
import requests as req

import time

In [2]:
#Cookies 
options=Options()
options.add_argument(r'user-data-dir=C:\Users\matia\anaconda3\Lib\site-packages\selenium\cookies') #Cargamos las cookies

In [3]:
url = 'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query=abner'

PATH = 'driver/chromedriver.exe'

In [6]:
driver = webdriver.Chrome(PATH, options=options)

driver.get(url)

In [7]:
#Scrapeo para sacar el valor de un jugador
driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text

'7,00 mill. €'

In [5]:
def valor(futbolista, path):
    try:
        driver = webdriver.Chrome(path)
        driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={futbolista}')
        
        # Espera hasta que el elemento esté presente
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink'))
        )
        
        try:
            valor = driver.find_element(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink').text
        except NoSuchElementException:
            valor = '0'
            print(f'No se encontró el valor de {futbolista}')

    except WebDriverException as e:
        valor = '0'
        print(f'Error con WebDriver al procesar {futbolista}: {e}')
    finally:
        driver.quit()

    return valor

In [6]:
#Hago una funcion para scrapear el valor de todos mis jugadores
'''def valor(futbolista,PATH):
    
    driver = webdriver.Chrome(PATH) 
    time.sleep(2)
    driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={futbolista}')
    time.sleep(2)
    try:
        valor = driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text
    except:
        valor = '0'
        print(f'No se encontro el valor de {futbolista}')

    driver.quit()
    return valor
'''

In [8]:
#Pruebo la funcion
valor('Carvajal',PATH)

'12,00 mill. €'

# Se lo aplico a mi tabla completa

In [9]:
stats2  = pd.read_csv('../../csv/csv_limpios/stats.csv')

In [10]:
lista_jugadores = stats2['Jugador'].tolist()

In [11]:
with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores]
    resultados = [f.result() for f in concurrent.futures.as_completed(futures)]

In [12]:
stats2['Valor Mercado'] = resultados

In [18]:
#Voy a buscar los jugadores que tienen valor 0 
stats2[stats2['Valor Mercado'] == '-']

,Unnamed: 0,Jugador,País,Posc,Equipo,Edad,Nacimiento,PJ,Titular,Mín,Gls.,Asistencias,Goles+Asistencias,Tarjetas Amarillas,Tarjetas Rojas,Goles Esperados,Asistencias Esperadas,Valor Mercado
17,17,Ángel Algobia,ESP,Centrocampista,Getafe,23,1999,26,11,1138,0,0,0,3,0,0.3,0.6,-
27,27,Marcos André,BRA,Delantero,Valencia,25,1996,17,3,447,1,0,1,2,1,1.5,0.4,-
30,30,José Antonio,ESP,Centrocampista,Cádiz,18,2004,2,0,46,0,1,1,0,0,0.0,0.0,-
37,37,Ricard Artero,ESP,Centrocampista,Girona,19,2003,7,0,133,0,0,0,0,0,0.5,0.0,-
57,57,Rodrigo Battaglia,ARG,Centrocampista,Mallorca,31,1991,15,8,686,0,0,0,5,0,0.2,0.0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3315,3315,Thomas Vermaelen,BEL,Defensa,Barcelona,31,1985,14,12,1018,0,1,1,3,0,0.3,0.4,-
3324,3324,Dídac Vilà,ESP,Defensa,Espanyol,28,1989,14,7,731,0,0,0,3,0,0.4,0.9,-
3326,3326,Vitolo,ESP,Centrocampista,Atlético Madrid,27,1989,14,8,618,1,2,3,1,1,1.4,0.6,-
3334,3334,Simone Zaza,ITA,Delantero,Valencia,26,1991,33,23,2077,13,1,14,9,1,10.8,2.1,-


# Lo aplico a temporada 2022


In [ ]:
df = pd.read_csv('../../csv/csv_limpios/jugadores_2022_lp.csv')

In [ ]:
lista_jugadores_22 = df['Jugador'].tolist()

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores_22]
    resultados = [f.result() for f in concurrent.futures.as_completed(futures)]

In [ ]:
df['Valor Mercado'] = resultados

# Exporto a csv limpio

In [13]:
stats2.to_csv('../../csv/csv_limpios/stats_completo.csv', index=False)